# Test the NCF module under folder [cf_ec2](../cf_ec2) with ml-1m dataset, save the best model (using integrated modules with compile and fit components, with gmf and mlp pretrain)

#### 5/4/2020

In [1]:
import numpy as np 
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import (
    Adam,
    Adamax,
    Adagrad,
    SGD,
    RMSprop
)
from tensorflow.keras.layers import (
    Embedding, 
    Input,
    Flatten, 
    Multiply, 
    Concatenate,
    Dense
)

import sys
sys.path.append('../')
from cf_ec2 import (
    GMF,
    MLP,
    NCF,
    Data,
    evaluation,
    evaluation_grouped
)

## step 1: load the data

In [2]:
train = pd.read_csv('../data/ml-1m-all-train.csv',sep=',',header=0,names=['user','item','rating','event_ts'])
test = pd.read_csv('../data/ml-1m-all-test.csv',sep=',',header=0,names=['user','item','rating','event_ts'])

In [3]:
test.user.nunique(), test.shape

(6040, (250088, 4))

In [4]:
train.user.nunique(), train.shape

(6040, (750121, 4))

In [5]:
train.head(3)

,user,item,rating,event_ts
0,1,3186,4.0,978300019
1,1,1721,4.0,978300055
2,1,1270,5.0,978300055


In [6]:
test.head(3)

,user,item,rating,event_ts
0,1,1545,4.0,978824139
1,1,527,5.0,978824195
2,1,745,3.0,978824268


In [7]:
test.rating.unique()

array([4., 5., 3., 2., 1.])

## step 2: prepare the data for gmf model training

In [8]:
%%time

dataset = Data(
    train=train,
    test=test,
    col_user='user',
    col_item='item',
    col_rating='rating',
    col_time='event_ts',
    binary=True,
    n_neg=4,
    n_neg_test=100,
    n_neg_limit=0
)
dataset.prepTrainDNN(negSample=True)
dataset.prepTestDNN(group=False)

CPU times: user 1min 18s, sys: 1.77 s, total: 1min 19s
Wall time: 1min 20s


In [9]:
dataset.interaction_train.head(3)

,user,item_interacted,item_negative
0,0,"{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","{40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 5..."
1,1,"{7, 11, 16, 17, 20, 28, 36, 40, 41, 42, 43, 44...","{0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15..."
2,2,"{130, 131, 132, 133, 134, 135, 136, 9, 137, 13...","{0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14..."


#### prepare the test dataset

In [10]:
newItems = set(dataset.items_test)-set(dataset.items)
idx2del = []
for idx,item in enumerate(dataset.items_test):
    if item in newItems:
        idx2del.append(idx)

length_test_original = len(dataset.users_test)
dataset.users_test = [
    dataset.users_test[idx]
    for idx in range(length_test_original) if idx not in idx2del
]
dataset.items_test = [
    dataset.items_test[idx]
    for idx in range(length_test_original) if idx not in idx2del
]
dataset.ratings_test = [
    dataset.ratings_test[idx]
    for idx in range(length_test_original) if idx not in idx2del
]

## step 3: create the model architecture and fit model with training data

In [11]:
train.user.nunique(), train.item.nunique()

(6040, 3660)

In [12]:
n_users = 6040
n_items = 3660
n_factors_gmf = 8
layers_mlp = [64,32,16,8]
# n_factors_gmf = 4
# layers_mlp = [16,8,4]
reg_gmf = 0.
reg_layers_mlp = [0.,0.,0.,0.]
learning_rate = 0.001
flg_pretrain = ''
filepath = ''
filepath_mlp_pretrain = ''
filepath_mlp_pretrain = ''
num_epochs = 20
batch_size = 512

ncf = NCF(
    n_users=n_users,
    n_items=n_items,
    n_factors_gmf=n_factors_gmf,
    layers_mlp=layers_mlp,
    reg_gmf=reg_gmf,
    reg_layers_mlp=reg_layers_mlp
)
ncf.create_model()

In [13]:
ncf.compile(learning_rate=learning_rate)

In [14]:
ncf.model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_mlp_User (Embedding)  (None, 1, 32)        193280      user_input[0][0]                 
__________________________________________________________________________________________________
embedding_mlp_Item (Embedding)  (None, 1, 32)        117120      item_input[0][0]                 
______________________________________________________________________________________________

In [15]:
hist = ncf.fit(
    dataset=dataset,
    batch_size=batch_size,
    num_epochs=num_epochs,
    path_model_weights='/Users/xyin/Documents/work/projects/rec_utils/metadata/ncf5/ncf-weights-improvement-{epoch:02d}-{val_loss:.4f}.hdf5',
    path_csvlog='/Users/xyin/Documents/work/projects/rec_utils/metadata/ncf5/ncf_log.csv'
)

Train on 3750605 samples, validate on 25252525 samples
Epoch 1/20
TimeHistory: 2.48 seconds, 103259.69 examples/second between steps 0 and 500
TimeHistory: 1.62 seconds, 157905.65 examples/second between steps 500 and 1000
TimeHistory: 1.61 seconds, 159429.32 examples/second between steps 1000 and 1500
TimeHistory: 1.64 seconds, 155770.93 examples/second between steps 1500 and 2000
TimeHistory: 1.64 seconds, 155922.44 examples/second between steps 2000 and 2500
TimeHistory: 1.60 seconds, 160065.15 examples/second between steps 2500 and 3000
TimeHistory: 1.64 seconds, 156305.24 examples/second between steps 3000 and 3500
TimeHistory: 1.66 seconds, 154308.99 examples/second between steps 3500 and 4000
TimeHistory: 1.61 seconds, 158697.51 examples/second between steps 4000 and 4500
TimeHistory: 1.66 seconds, 154343.70 examples/second between steps 4500 and 5000
TimeHistory: 1.64 seconds, 156344.57 examples/second between steps 5000 and 5500
TimeHistory: 1.59 seconds, 161432.12 examples/se

TimeHistory: 1.74 seconds, 147494.42 examples/second between steps 40000 and 40500
TimeHistory: 1.69 seconds, 151247.33 examples/second between steps 40500 and 41000
TimeHistory: 1.65 seconds, 154757.22 examples/second between steps 41000 and 41500
TimeHistory: 1.74 seconds, 146906.14 examples/second between steps 41500 and 42000
TimeHistory: 1.75 seconds, 145968.92 examples/second between steps 42000 and 42500
TimeHistory: 1.74 seconds, 146851.41 examples/second between steps 42500 and 43000
TimeHistory: 1.70 seconds, 150963.97 examples/second between steps 43000 and 43500
global_steps: 43956, last_log_step: 43500

Epoch 00006: val_loss did not improve from 0.13965
3750605/3750605 - 71s - loss: 0.2426 - accuracy: 0.8956 - val_loss: 0.1498 - val_accuracy: 0.9425
Epoch 7/20
TimeHistory: 46.91 seconds, 5457.81 examples/second between steps 43500 and 44000
TimeHistory: 1.53 seconds, 167080.08 examples/second between steps 44000 and 44500
TimeHistory: 1.49 seconds, 171610.30 examples/secon

TimeHistory: 1.69 seconds, 151135.79 examples/second between steps 81500 and 82000
TimeHistory: 1.52 seconds, 168067.64 examples/second between steps 82000 and 82500
TimeHistory: 1.55 seconds, 164972.47 examples/second between steps 82500 and 83000
TimeHistory: 1.57 seconds, 163001.69 examples/second between steps 83000 and 83500
TimeHistory: 1.38 seconds, 186055.58 examples/second between steps 83500 and 84000
TimeHistory: 1.52 seconds, 168034.48 examples/second between steps 84000 and 84500
TimeHistory: 1.56 seconds, 163731.25 examples/second between steps 84500 and 85000
TimeHistory: 1.54 seconds, 166127.32 examples/second between steps 85000 and 85500
TimeHistory: 1.53 seconds, 167563.83 examples/second between steps 85500 and 86000
TimeHistory: 1.56 seconds, 164504.41 examples/second between steps 86000 and 86500
TimeHistory: 1.54 seconds, 165912.49 examples/second between steps 86500 and 87000
TimeHistory: 1.56 seconds, 163987.68 examples/second between steps 87000 and 87500
glob

TimeHistory: 1.57 seconds, 163259.34 examples/second between steps 123000 and 123500
TimeHistory: 1.69 seconds, 151634.87 examples/second between steps 123500 and 124000
TimeHistory: 1.79 seconds, 143013.26 examples/second between steps 124000 and 124500
global_steps: 124542, last_log_step: 124500

Epoch 00017: val_loss did not improve from 0.13289
3750605/3750605 - 73s - loss: 0.1948 - accuracy: 0.9187 - val_loss: 0.1568 - val_accuracy: 0.9391
Epoch 18/20
TimeHistory: 48.72 seconds, 5254.19 examples/second between steps 124500 and 125000
TimeHistory: 1.57 seconds, 163094.71 examples/second between steps 125000 and 125500
TimeHistory: 1.73 seconds, 148029.14 examples/second between steps 125500 and 126000
TimeHistory: 1.97 seconds, 130026.47 examples/second between steps 126000 and 126500
TimeHistory: 1.68 seconds, 152601.15 examples/second between steps 126500 and 127000
TimeHistory: 1.91 seconds, 134214.89 examples/second between steps 127000 and 127500
TimeHistory: 1.97 seconds, 130

#### try to load the parameters from the best model

In [16]:
ncf.model.load_weights('../metadata/ncf5/ncf-weights-improvement-12-0.1329.hdf5')

In [18]:
ncf.model.save('../metadata/ncf5/ncf-best.hdf5')

In [19]:
len(dataset.users_test), len(dataset.items_test)

(25252525, 25252525)

In [22]:
len(set(dataset.users_test)), len(set(dataset.items_test))

(6040, 3660)

In [23]:
dataset.ratings_test[-5:]

[0.0, 0.0, 0.0, 0.0, 0.0]

In [24]:
rating_true = pd.DataFrame({
    'user':dataset.users_test,
    'item':dataset.items_test,
    'rating':dataset.ratings_test
})
rating_true.head(3)

,user,item,rating
0,0,171,1.0
1,0,110,0.0
2,0,2994,0.0


In [25]:
rating_true.shape, rating_true[rating_true.rating>0].shape

((25252525, 3), (250025, 3))

In [27]:
%%time

list_preds = ncf.model.predict(
        x=[
            np.array(dataset.users_test),
            np.array(dataset.items_test)
        ]
    ).flatten()

CPU times: user 10min 6s, sys: 1min 17s, total: 11min 24s
Wall time: 7min 59s


In [28]:
len(list_preds)

25252525

In [29]:
rating_pred = pd.DataFrame({
    'user':dataset.users_test,
    'item':dataset.items_test,
    'rating':list_preds
})
rating_pred.head(3)

,user,item,rating
0,0,171,0.932909
1,0,110,0.931356
2,0,2994,0.016190


In [30]:
rating_pred = rating_pred.rename(columns={'rating':'prediction'})

In [32]:
%%time

ndcg = evaluation.ndcg_at_k(
    rating_true.loc[rating_true.rating>0],
    rating_pred,
    col_user='user',
    col_item='item',
    col_rating='rating',
    col_pred='prediction',
    k=10
)

recall = evaluation.recall_at_k(
    rating_true.loc[rating_true.rating>0],
    rating_pred,
    col_user='user',
    col_item='item',
    col_rating='rating',
    col_pred='prediction',
    k=10
)

precision = evaluation.precision_at_k(
    rating_true.loc[rating_true.rating>0],
    rating_pred,
    col_user='user',
    col_item='item',
    col_rating='rating',
    col_pred='prediction',
    k=10
)

CPU times: user 43.7 s, sys: 6.16 s, total: 49.8 s
Wall time: 50.4 s


In [33]:
ndcg, recall, precision

(0.18303133620423187, 0.10581144589480436, 0.15637417218543048)